In [ ]:
# imports here

Distinguish pits by snow depth classes: 'shallow', 'medium', 'deep'

PLOT: #1 Density

  - depth (cm) vs. density (kg/m3) for snow depth classes
  - 3 colors: shallow (r), medium (g), deep (b)
  - vertical bars to show the 10cm measurement increments (see ../images)  
  - time period -- (hmm, all? single day? 1 week avg?)

PLOT: #2 Temperature

  - depth (cm) vs. temperature (C) for snow depth classes
  - 3 colors: shallow (r), medium (g), deep (b)
  - x-axis is -10 to 0 (see ../images)  
  - time period -- (hmm, all? single day? 1 week avg?)
  - could plot all pits with light gray line (or group by depth classes in a light/transparent color) and then take the avg. of 'shallow', 'medium', 'deep' and plot with a bold color line

PLOT: #3
  - do something with stratigraphy here....still thinking on that...